In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [3]:
!nvidia-smi

Fri Dec 26 12:26:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:17:00.0 Off |                    0 |
| N/A   60C    P0             75W /  300W |    3713MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Prompting/responses/ready_for_metrics


In [5]:
labse_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
labse_model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
muril_model = AutoModel.from_pretrained("google/muril-base-cased")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labse_model.to(device)
muril_model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(197285, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [12]:
!pip install transformers[sentencepiece]
!pip install evaluate
!pip install bert-score
!pip install unbabel-comet
!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 11.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 24.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
import numpy as np
from evaluate import load
from sentence_transformers import SentenceTransformer, util

def compute_translation_scores(predictions, references, lang='en'):
    """
    Compute reference-based translation evaluation scores using:
    1. BERTScore (MuRIL)
    2. LaBSE cosine similarity
    
    Args:
        predictions (list of str): Model outputs
        references (list of str): Reference translations
        lang (str): Language code for BERTScore ('en', 'hi', etc.)
    
    Returns:
        dict: {'muril_f1': float, 'labse_cosine': float}
    """
    assert len(predictions) == len(references), "Predictions and references must have the same length"

    # ---------- BERTScore (MuRIL) ----------
    bertscore = load("bertscore")
    bert_results = bertscore.compute(
        predictions=predictions,
        references=references,
        model_type='google/muril-base-cased',
        num_layers=4,
        lang=lang
    )
    muril_f1_mean = float(np.mean(bert_results['f1']))

    # ---------- LaBSE Cosine Similarity ----------
    model = SentenceTransformer('sentence-transformers/LaBSE')
    pred_emb = model.encode(predictions, convert_to_tensor=True)
    ref_emb = model.encode(references, convert_to_tensor=True)
    cosine_sim_matrix = util.cos_sim(pred_emb, ref_emb)
    # Take diagonal (each prediction with its reference)
    labse_cosine_mean = float(cosine_sim_matrix.diag().mean())

    return {'muril_score': muril_f1_mean, 'labse_cosine': labse_cosine_mean}

In [6]:
# def compute_ref_based_scores(mt_texts, ref_texts, method="max"):
#     results = []
    
#     for i, mt in enumerate(mt_texts):
#         ref = ref_texts[i]
        
#         # Skip if empty or NaN
#         if not mt or not ref or pd.isna(mt) or pd.isna(ref):
#             continue
        
#         # --- LaBSE ---
#         try:
#             inputs = labse_tokenizer(ref, mt, padding=True, truncation=True, return_tensors="pt").to(device)
#             with torch.no_grad():
#                 embeddings = labse_model(**inputs).pooler_output
#             if embeddings.shape[0] >= 2:
#                 labse_ref_mt =  F.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0)).item()
#             else:
#                 labse_ref_mt = 0.0
#         except Exception as e:
#             print(f"Skipping LaBSE pair due to error: {e}")
#             labse_ref_mt = 0.0

#         # --- MuRIL ---
#         try:
#             inputs = muril_tokenizer(ref, mt, padding=True, truncation=True, return_tensors="pt").to(device)
#             with torch.no_grad():
#                 outputs = muril_model(**inputs)
#                 embeddings = outputs.last_hidden_state.mean(dim=1)
#             if embeddings.shape[0] >= 2:
#                 muril_ref_mt =  F.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0)).item()
#             else:
#                 muril_ref_mt = 0.0
#         except Exception as e:
#             print(f"Skipping MuRIL pair due to error: {e}")
#             muril_ref_mt = 0.0

#         results.append({
#             "mt": mt,
#             "labse_ref_mt": labse_ref_mt,
#             "muril_ref_mt": muril_ref_mt
#         })
    
#     df_scores = pd.DataFrame(results)
    
#     # System-level averages
#     system_scores = {
#         "labse_ref_mt": df_scores["labse_ref_mt"].mean() if not df_scores.empty else 0.0,
#         "muril_ref_mt": df_scores["muril_ref_mt"].mean() if not df_scores.empty else 0.0
#     }
    
#     return system_scores

In [7]:
labse = {}
muril = {}

## Original

In [8]:
file_name = "llama_original.csv"

In [9]:
df = pd.read_csv(file_name)
df.head()

,Sentence,Response,sent_meant,gt
0,"Chanting, the choir raised the volume as the c...","श्रद्धांजली घेतल्यानंतर, गायिका घोषवाण्याने प्...","Chanting, the choir raised the volume as the c...","धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ..."
1,A six-month-old calf was submitted for examina...,एकूण सहा महिन्यांचा वाढतोळा ज्याला परीक्षेसाठी...,A six-month-old calf was submitted for examina...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...
2,Planning authorities should provide alternativ...,व्यवस्थापन अधिकारी लहान व्यवसायासाठी घरगुती क्...,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...
3,"As the machine develops, the forms we use to r...","येत्या वेळी मशीन विकसित होत असताना, आम्ही पूर्...","As the machine develops, the forms we use to r...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प..."
4,"As mentioned first, impressions can be mislead...",पहिल्या वेळी सांगितल्याप्रमाणे भावना वाईट वाटू...,"As mentioned first, impressions can be mislead...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते."


In [14]:
predictions = df['Response'].tolist()
references = df['gt'].tolist()

    
scores = compute_translation_scores(predictions, references, lang='en')
print("Scores:", scores)

Scores: {'muril_score': 0.6535316021354111, 'labse_cosine': 0.8404159545898438}


In [16]:
import os
import pandas as pd
# CSV folder is current directory
csv_folder = "."

# Collect results
results = []

for file_name in os.listdir(csv_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
        df = pd.read_csv(file_path)

        predictions = df['Response'].tolist()
        references = df['gt'].tolist()

        # Compute translation scores
        scores = compute_translation_scores(predictions, references, lang='en')

        # Append results with file info
        results.append({
            "file": file_name,
            "strategy": file_name.split('_')[1],  # part after first '_'
            "scores": scores
        })

# Sort results by strategy (the word after the first '_')
results_sorted = sorted(results, key=lambda x: x['strategy'])

# Print sorted results
for r in results_sorted:
    print(f"File: {r['file']}, Strategy: {r['strategy']}, Scores: {r['scores']}")

File: sarvam_few_cot.csv, Strategy: few, Scores: {'muril_score': 0.6709941561315574, 'labse_cosine': 0.8508744239807129}
File: llama_few_cot.csv, Strategy: few, Scores: {'muril_score': 0.6120327737401513, 'labse_cosine': 0.8194451332092285}
File: llama_few_zero.csv, Strategy: few, Scores: {'muril_score': 0.619218980272611, 'labse_cosine': 0.8216394186019897}
File: gemma_few_cot.csv, Strategy: few, Scores: {'muril_score': 0.7056032255584118, 'labse_cosine': 0.8925914168357849}
File: gemma_few_zero.csv, Strategy: few, Scores: {'muril_score': 0.7013319251584071, 'labse_cosine': 0.8882648944854736}
File: sarvam_few_zero.csv, Strategy: few, Scores: {'muril_score': 0.70191195899365, 'labse_cosine': 0.8803153038024902}
File: llama_original.csv, Strategy: original.csv, Scores: {'muril_score': 0.6535316021354111, 'labse_cosine': 0.8404159545898438}
File: sarvam_original.csv, Strategy: original.csv, Scores: {'muril_score': 0.655046463796967, 'labse_cosine': 0.8422093391418457}
File: gemma_origin

In [15]:
# predictions = df['Response'].tolist()
# references = df['gt'].tolist()

# print(predictions[:5])
# print(references[:5])

# scores = compute_ref_based_scores(predictions, references)
# labse = scores['labse_ref_mt']
# muril= scores['muril_ref_mt']

# print(f"LaBSE and MuRIL Scores for {file_name}:")
# print(labse, muril)

## Only Punctuations

In [7]:
file_name = "with_outputs.csv"
mode = "with"

In [8]:
df = pd.read_csv(file_name)

predictions = df['prediction_mar'].tolist()
references = df['gt_mar'].tolist()

scores = compute_ref_based_scores(predictions, references)
labse[mode] = scores['labse_ref_mt']
muril[mode] = scores['muril_ref_mt']

print(f"LaBSE and MuRIL Scores for {mode}:")
print(scores)

LaBSE and MuRIL Scores for with:
{'labse_ref_mt': 0.9197827294812637, 'muril_ref_mt': 0.9981659788903645}


## Without Punctuations

In [9]:
file_name = "without_outputs.csv"
mode = "without"

In [10]:
df = pd.read_csv(file_name)

predictions = df['prediction_mar'].tolist()
references = df['gt_mar'].tolist()

scores = compute_ref_based_scores(predictions, references)
labse[mode] = scores['labse_ref_mt']
muril[mode] = scores['muril_ref_mt']

print(f"LaBSE and MuRIL Scores for {mode}:")
print(scores)

LaBSE and MuRIL Scores for without:
{'labse_ref_mt': 0.9201303059861079, 'muril_ref_mt': 0.9981740449472081}


## Combined - LR & Epochs Changed

In [11]:
file_name = "combined_2x_outputs.csv"
mode = "combined_2x"

In [12]:
df = pd.read_csv(file_name)

predictions = df['prediction_mar'].tolist()
references = df['gt_mar'].tolist()

scores = compute_ref_based_scores(predictions, references)
labse[mode] = scores['labse_ref_mt']
muril[mode] = scores['muril_ref_mt']

print(f"LaBSE and MuRIL Scores for {mode}:")
print(scores)

LaBSE and MuRIL Scores for combined_2x:
{'labse_ref_mt': 0.9204223514679323, 'muril_ref_mt': 0.9981520064735604}


## Combined - LR & Epochs & Dataset Changed

In [13]:
file_name = "combined_x_outputs.csv"
mode = "combined_x"

In [14]:
df = pd.read_csv(file_name)

predictions = df['prediction_mar'].tolist()
references = df['gt_mar'].tolist()

scores = compute_ref_based_scores(predictions, references)
labse[mode] = scores['labse_ref_mt']
muril[mode] = scores['muril_ref_mt']

print(f"LaBSE and MuRIL Scores for {mode}:")
print(scores)

LaBSE and MuRIL Scores for combined_x:
{'labse_ref_mt': 0.9217669360018302, 'muril_ref_mt': 0.9981642475458181}


## T5

In [15]:
file_name = "t5_outputs.csv"
mode = "t5"

In [16]:
df = pd.read_csv(file_name)

predictions = df['prediction_mar'].tolist()
references = df['gt_mar'].tolist()

scores = compute_ref_based_scores(predictions, references)
labse[mode] = scores['labse_ref_mt']
muril[mode] = scores['muril_ref_mt']

print(f"LaBSE and MuRIL Scores for {mode}:")
print(scores)

LaBSE and MuRIL Scores for t5:
{'labse_ref_mt': 0.9212691990159818, 'muril_ref_mt': 0.9981615691031951}


In [17]:
for key, val in labse.items():
    print(f"{key} : {val:.4f}")

original : 0.9226
with : 0.9198
without : 0.9201
combined_2x : 0.9204
combined_x : 0.9218
t5 : 0.9213


In [21]:
for key, val in muril.items():
    print(f"{key} : {val:.6f}")

original : 0.998117
with : 0.998166
without : 0.998174
combined_2x : 0.998152
combined_x : 0.998164
t5 : 0.998162
